<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/autogen/gemini/gemini_custom_assitant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=GOOGLE_API_KEY

In [3]:
import os
os.environ['GOOGLE_API_KEY']=GOOGLE_API_KEY
os.environ['GEMINI_API_KEY']=GEMINI_API_KEY

In [5]:
%pip install -U -q autogen autogen_ext[openai] google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.4/306.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.8 MB/s eta 0:00:00


In [13]:
import autogen
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent
from autogen_ext.models import *

In [7]:
genai.configure(api_key=GEMINI_API_KEY)

In [18]:
gemini_flash="gemini-2.0-flash"
gemma_1b="gemma-3-1b-it"
gemma_4b="gemma-3-4b-it"
gemma_12b="gemma-3-12b-it"
gemma_27b="gemma-3-27b-it"
gemma_e4b="gemma-3n-e4b-it"
llama3="llama3.2:1b"

In [19]:
def ask_gemini(prompt, model=gemini_flash):
    response = genai.GenerativeModel(model).generate_content(prompt)
    return response.text

In [26]:
# 3. Create Custom Agent
# 3. Create Custom Agent with Corrected Method Signature
class GeminiAgent(AssistantAgent):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)

    def generate_reply(self, messages=None, sender=None, **kwargs):
        # Extract conversation history
        if messages is None:
            messages = self.chat_messages[sender]

        # Format for Gemini (simple concatenation)
        prompt = "\n".join([msg["content"] for msg in messages])

        # Get Gemini response
        response = ask_gemini(prompt)
        return response
# 4. Setup Agents
gemini_assistant = GeminiAgent(
    name="Gemini_Expert",
    system_message="""You are a helpful assistant powered by Google Gemini. When you have fully addressed the user's request,
    end your response with 'TERMINATE'.""",
)

In [27]:
user_proxy = UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",  # Auto-reply
    max_consecutive_auto_reply=3,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "coding"},
)


In [28]:
# 5. Start Conversation
user_proxy.initiate_chat(
    gemini_assistant,
    message="Explain quantum computing like I'm 10 years old. "
)

User_Proxy (to Gemini_Expert):

Explain quantum computing like I'm 10 years old

--------------------------------------------------------------------------------
Gemini_Expert (to User_Proxy):

Okay, imagine regular computers like light switches. They can be either ON (which we call 1) or OFF (which we call 0).  Everything a computer does is built using these on/off switches.

Quantum computers are like magical light switches that can be BOTH ON AND OFF AT THE SAME TIME!  It's like a light switch that's flickering in between.  We call this "superposition."

Think of it like this:

*   **Regular switch (Regular computer):**  It can be pointing either UP (1) or DOWN (0).
*   **Quantum switch (Quantum computer):** It can be pointing UP, DOWN, OR even SPINNING!  It's in a blurry, mixed-up state until you look at it. When you look, it *has* to decide to be either UP or DOWN, like a regular switch.

These "quantum switches" are called **qubits**. Because qubits can be in so many states at on

ChatResult(chat_id=None, chat_history=[{'content': "Explain quantum computing like I'm 10 years old", 'role': 'assistant', 'name': 'User_Proxy'}, {'content': 'Okay, imagine regular computers like light switches. They can be either ON (which we call 1) or OFF (which we call 0).  Everything a computer does is built using these on/off switches.\n\nQuantum computers are like magical light switches that can be BOTH ON AND OFF AT THE SAME TIME!  It\'s like a light switch that\'s flickering in between.  We call this "superposition."\n\nThink of it like this:\n\n*   **Regular switch (Regular computer):**  It can be pointing either UP (1) or DOWN (0).\n*   **Quantum switch (Quantum computer):** It can be pointing UP, DOWN, OR even SPINNING!  It\'s in a blurry, mixed-up state until you look at it. When you look, it *has* to decide to be either UP or DOWN, like a regular switch.\n\nThese "quantum switches" are called **qubits**. Because qubits can be in so many states at once, quantum computers c